# 0.0 Imports

In [2]:
import re 
import sqlite3 
import requests
import numpy as np
import pandas as pd

from bs4        import BeautifulSoup
from datetime   import datetime
from sqlalchemy import create_engine

# 1.0 Loading Data

In [3]:
# URL 
url = 'https://www2.hm.com/en_us/men/products/jeans.html'
    
# Parameters
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5),AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

# Request to URL 
page = requests.get( url, headers=headers )

# Beautiful Soup object
soup = BeautifulSoup( page.text, 'html.parser' )


## 1.1 Data Collection

In [4]:
# ==============================product data========================================

products = soup.find( 'ul', class_='products-listing small' )

product_list = products.find_all( 'article', class_='hm-product-item')

# product id
product_id = [p.get( 'data-articlecode' ) for p in product_list]

# product category
product_category = [p.get( 'data-category' ) for p in product_list]

# product name
product_list = products.find_all( 'a', class_='link' )
product_name = [p.get_text() for p in product_list]

# price
product_list = products.find_all( 'span', class_='price regular' )
product_price = [p.get_text() for p in product_list]

data = pd.DataFrame( [product_id, product_category, product_name, product_price] ).T
data.columns = ['product_id', 'product_category', 'product_name', 'product_price']
data.head()



,product_id,product_category,product_name,product_price
0,0985159001,men_jeans_skinny,Skinny Jeans,$ 24.99
1,1024256001,men_jeans_slim,Slim Jeans,$ 19.99
2,1008110001,men_jeans_slim,Freefit® Slim Jeans,$ 49.99
3,1024256002,men_jeans_slim,Slim Jeans,$ 19.99
4,0979945001,men_jeans_loose,Loose Jeans,$ 39.99


In [5]:
data.shape

(36, 4)

## 1.2 Data Collection by Product

In [6]:
# empty dataframe
df_compositions = pd.DataFrame() 

# unique columns for all products
aux = []

df_pattern = pd.DataFrame( columns=['Art. No.', 'Composition', 'Fit', 'Product safety', 'Size'] )
for i in range ( len( data ) ):
    # API Requests
    url = 'https://www2.hm.com/en_us/productpage.' + data.loc[i, 'product_id' ] + '.html'
    #url = 'https://www2.hm.com/en_us/productpage.' + '0985159007' + '.html'
    print ( 'Product: {}'.format( url )  )
        
    page = requests.get( url, headers=headers )

    # Beautiful Soup object
    soup = BeautifulSoup( page.text, 'html.parser' )
    
    
    
    # ================================= color name =================================
    product_list = soup.find_all( 'a', class_='filter-option miniature active' ) + soup.find_all('a', class_='filter-option miniature')
    color_name = [p.get( 'data-color') for p in product_list]

    # product id 
    product_id = [p.get( 'data-articlecode' ) for p in product_list]

    df_color = pd.DataFrame( [product_id, color_name] ).T
    df_color.columns = ['product_id', 'color_name']
    
    for j in range( len( df_color ) ):
        # API Requests
        url = 'https://www2.hm.com/en_us/productpage.' + df_color.loc[j, 'product_id' ] + '.html'
        print ( 'Color: {}'.format( url )  )
        
        page = requests.get( url, headers=headers )

        # Beautiful Soup object
        soup = BeautifulSoup( page.text, 'html.parser' )
        
       
        #================================= Product Name =================================
        product_name = soup.find_all('section', class_='product-name-price')
        product_name = product_name[0].get_text()
        #print( product_name )                                        
                                                                              
              
        # ================================= Product Price =================================
        product_price = soup.find_all( 'div', class_='primary-row product-item-price' )
        #product_price = re.findall( r'\d+\.?d+', product_price[0].get_text() )[0]
        product_price = product_price[0].get_text()
        #print( product_price )                                                         
                                                                 
        
        # ================================= composition =================================

        # HTML all data stored 
        product_composition_list = soup.find_all( 'div', class_='details-attributes-list-item' )
        product_composition = [list(filter(None, p.get_text().split( '\n' ) ) ) for p in product_composition_list]

        # Collect the size here                             
        size_text = soup.find_all('dd')[0].text
        size_text = size_text if 'cm' in size_text else pd.NA

        for i in range(len(product_composition)):
            if product_composition[i][0] == 'Size':
                product_composition[i] = ['Size', size_text]
                break
        else:
            product_composition.append(['Size', size_text])

                
        # rename columns DataFrame
        df_composition = pd.DataFrame(product_composition).T
        df_composition.columns = df_composition.iloc[0]

        # delete first row and substitution null values 
        df_composition = df_composition.iloc[1:].fillna( method='ffill' )
                
        # remove pocket lining, shell and lining 
        df_composition['Composition'] = df_composition['Composition'].replace( 'Pocket lining:', '', regex=True )
        df_composition['Composition'] = df_composition['Composition'].replace( 'Shell:', '', regex=True )
        df_composition['Composition'] = df_composition['Composition'].replace( 'Lining:', '', regex=True )
        
        # garantee the same number of columns
        df_composition = pd.concat( [ df_pattern, df_composition ], axis=0)
        
                
        df_composition = df_composition[['Art. No.', 'Composition', 'Fit', 'Product safety', 'Size']]
        
                  
        # remane columns 
        df_composition.columns = ['product_id', 'composition', 'fit', 'product_safety', 'size']
        df_compositions['product_name'] = product_name        
        df_compositions['product_price'] = product_price
        
        
        # keep new columns if it shows up 
        aux = aux + df_composition.columns.tolist()
        
                 
        # merge data color + data compostition
        df_composition = pd.merge( df_composition, df_color,how='left', on='product_id' )
        
                       
        # all products
        df_compositions = pd.concat ( [df_compositions, df_composition], axis=0 )
        
        
# join showroom data + details
df_compositions['style_id'] = df_compositions['product_id'].apply( lambda x: x[:-3] )
df_compositions['color_id'] = df_compositions['product_id'].apply( lambda x: x[-3:] )


# scrapy datetime
df_compositions['scrapy_datetime'] = datetime.now().strftime( '%Y-%m-%d %H:%M:%S')


Product: https://www2.hm.com/en_us/productpage.0985159001.html
Color: https://www2.hm.com/en_us/productpage.0985159001.html
Color: https://www2.hm.com/en_us/productpage.0985159002.html
Color: https://www2.hm.com/en_us/productpage.0985159003.html
Color: https://www2.hm.com/en_us/productpage.0985159004.html
Color: https://www2.hm.com/en_us/productpage.0985159005.html
Color: https://www2.hm.com/en_us/productpage.0985159006.html
Color: https://www2.hm.com/en_us/productpage.0985159007.html
Color: https://www2.hm.com/en_us/productpage.0985159008.html
Product: https://www2.hm.com/en_us/productpage.1024256001.html
Color: https://www2.hm.com/en_us/productpage.1024256001.html
Color: https://www2.hm.com/en_us/productpage.1024256002.html
Color: https://www2.hm.com/en_us/productpage.1024256003.html
Color: https://www2.hm.com/en_us/productpage.1024256004.html
Color: https://www2.hm.com/en_us/productpage.1024256005.html
Color: https://www2.hm.com/en_us/productpage.1024256006.html
Color: https://www2.

Color: https://www2.hm.com/en_us/productpage.0993887007.html
Product: https://www2.hm.com/en_us/productpage.1024256003.html
Color: https://www2.hm.com/en_us/productpage.1024256003.html
Color: https://www2.hm.com/en_us/productpage.1024256001.html
Color: https://www2.hm.com/en_us/productpage.1024256002.html
Color: https://www2.hm.com/en_us/productpage.1024256004.html
Color: https://www2.hm.com/en_us/productpage.1024256005.html
Color: https://www2.hm.com/en_us/productpage.1024256006.html
Color: https://www2.hm.com/en_us/productpage.1024256007.html
Color: https://www2.hm.com/en_us/productpage.1024256008.html
Product: https://www2.hm.com/en_us/productpage.0979945003.html
Color: https://www2.hm.com/en_us/productpage.0979945003.html
Color: https://www2.hm.com/en_us/productpage.0979945001.html
Color: https://www2.hm.com/en_us/productpage.0979945002.html
Color: https://www2.hm.com/en_us/productpage.0979945009.html
Color: https://www2.hm.com/en_us/productpage.0979945012.html
Color: https://www2.

Color: https://www2.hm.com/en_us/productpage.0938875008.html
Color: https://www2.hm.com/en_us/productpage.0938875010.html
Color: https://www2.hm.com/en_us/productpage.0938875012.html
Color: https://www2.hm.com/en_us/productpage.0938875013.html
Color: https://www2.hm.com/en_us/productpage.0938875014.html
Product: https://www2.hm.com/en_us/productpage.1004199002.html
Color: https://www2.hm.com/en_us/productpage.1004199002.html
Color: https://www2.hm.com/en_us/productpage.1004199003.html
Color: https://www2.hm.com/en_us/productpage.1004199004.html
Color: https://www2.hm.com/en_us/productpage.1004199005.html
Color: https://www2.hm.com/en_us/productpage.1004199007.html
Product: https://www2.hm.com/en_us/productpage.0690449056.html
Color: https://www2.hm.com/en_us/productpage.0690449056.html
Color: https://www2.hm.com/en_us/productpage.0690449001.html
Color: https://www2.hm.com/en_us/productpage.0690449002.html
Color: https://www2.hm.com/en_us/productpage.0690449006.html
Color: https://www2.

In [7]:
df_compositions.head()

,product_name,product_price,product_id,composition,fit,product_safety,size,color_name,style_id,color_id,scrapy_datetime
0,\n\n\nRelaxed Jeans\n\n\n\n\n\n\n\r\n ...,\n\r\n $39.99\n,0985159001,"Cotton 99%, Spandex 1%",Skinny fit,NaN,"The model is 185cm/6'1"" and wears a size 31/32",Black,0985159,001,2022-10-09 07:57:19
1,\n\n\nRelaxed Jeans\n\n\n\n\n\n\n\r\n ...,\n\r\n $39.99\n,0985159001,Cotton 100%,Skinny fit,NaN,"The model is 185cm/6'1"" and wears a size 31/32",Black,0985159,001,2022-10-09 07:57:19
2,\n\n\nRelaxed Jeans\n\n\n\n\n\n\n\r\n ...,\n\r\n $39.99\n,0985159001,Cotton 100%,Skinny fit,NaN,"The model is 185cm/6'1"" and wears a size 31/32",Black,0985159,001,2022-10-09 07:57:19
3,\n\n\nRelaxed Jeans\n\n\n\n\n\n\n\r\n ...,\n\r\n $39.99\n,0985159001,Cotton 100%,Skinny fit,NaN,"The model is 185cm/6'1"" and wears a size 31/32",Black,0985159,001,2022-10-09 07:57:19
4,\n\n\nRelaxed Jeans\n\n\n\n\n\n\n\r\n ...,\n\r\n $39.99\n,0985159001,Cotton 100%,Skinny fit,NaN,"The model is 185cm/6'1"" and wears a size 31/32",Black,0985159,001,2022-10-09 07:57:19


In [8]:
len(df_compositions['product_id'].unique() )

125

In [9]:
df_compositions.shape

(1777, 11)

# 2.0 Data Cleaning

In [10]:
# product id
df_data = df_compositions.dropna( subset=['product_id'] )

# product name
df_data['product_name'] = df_data['product_name'].str.replace( '\n', '' )
df_data['product_name'] = df_data['product_name'].str.replace( '\r', '' )
df_data['product_name'] = df_data['product_name'].str.replace( '\t', '' )
df_data['product_name'] = df_data['product_name'].str.replace( '  ', '' )
df_data['product_name'] = df_data['product_name'].str.replace( 'freefit®_', '' )
df_data['product_name'] = df_data['product_name'].str.replace( '_', '' )
df_data['product_name'] = df_data['product_name'].str.replace( ' ', '_' ).str.lower()
df_data['product_name'] = df_data['product_name'].str.split('$').str[0]


# produtc price
df_data['product_price'] = df_data['product_price'].str.replace( '\n', '' )
df_data['product_price'] = df_data['product_price'].str.replace( '\r', '' )
df_data['product_price'] = df_data['product_price'].str.replace( '\t', '' )
df_data['product_price'] = df_data['product_price'].str.replace( '$', '' )
df_data['product_price'] = df_data['product_price'].astype( float )

#  color name 
df_data['color_name'] = df_data['color_name'].str.replace( ' ', '_').str.lower()      
                                               
# fit
df_data['fit'] = df_data['fit'].apply( lambda x: x.replace( ' ', '_').lower() if pd.notnull( x ) else x )

# size number       # exclui a medida cm
df_data['size_number'] = df_data['size'].apply( lambda x: re.search( '\d{3}cm', x ).group(0) if pd.notnull( x ) else x)
df_data['size_number'] = df_data['size'].apply( lambda x: re.search( '\d+', x ).group(0) if pd.notnull( x ) else x)

# size model 
df_data['size_model'] = df_data['size'].str.extract( '(\d+/\\d+)' )

# break composition by comma
df1 = df_data['composition'].str.split( ',', expand=True ).reset_index(drop=True)


# cotton | polyester | spandex
df_ref = pd.DataFrame( index=np.arange( len( data ) ), columns=['cotton', 'polyester', 'spandex'] )

# ================================== composition ==================================================

# --------cotton-----------------
df_cotton_0 = df1.loc[df1[0].str.contains( 'Cotton', na=True ), 0]
df_cotton_0.name = 'cotton'

df_cotton_1 = df1.loc[df1[1].str.contains( 'Cotton', na=True ), 1]
df_cotton_1.name = 'cotton'

# combine 
df_cotton = df_cotton_0.combine_first( df_cotton_1 )

df_ref = pd.concat( [df_ref, df_cotton], axis=1 )
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated( keep='last' )]

# ----------polyester--------------
df_polyester_0 = df1.loc[df1[0].str.contains( 'Polyester', na=True ), 0]
df_polyester_0.name = 'polyester'

df_polyester_1 = df1.loc[df1[1].str.contains( 'Polyester', na=True ), 1]
df_polyester_1.name = 'polyester'

# combine 
df_polyester = df_polyester_0.combine_first( df_polyester_1 )

df_ref = pd.concat( [df_ref, df_polyester], axis=1 )
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated( keep='last')]
df_ref['polyester'] = df_ref['polyester'].fillna('Polyester 0%')

# ----------spandex--------------
df_spandex_1 = df1.loc[df1[1].str.contains( 'Spandex', na=True ), 1]
df_spandex_1.name = 'spandex'

df_spandex_2 = df1.loc[df1[2].str.contains( 'Spandex', na=True ), 2]
df_spandex_2.name = 'spandex'
                       
# combine
df_spandex = df_spandex_1.combine_first( df_spandex_2 )

df_ref = pd.concat( [df_ref, df_spandex], axis=1 )
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated( keep='last')]
df_ref['spandex'] = df_ref['spandex'].fillna('Spandex 0%')
  
# join of combine with product_id
df_aux = pd.concat( [df_data['product_id'].reset_index(drop=True), df_ref], axis=1 )
                       
                       
# format composition data 
df_aux['cotton']     = df_aux['cotton'].apply( lambda x: int( re.search( '\d+', x ).group(0) ) / 100 if pd.notnull( x ) else x )
df_aux['polyester']  = df_aux['polyester'].apply( lambda x: int( re.search( '\d+', x ).group(0) ) / 100 if pd.notnull( x ) else x )
df_aux['spandex']    = df_aux['spandex'].apply( lambda x: int( re.search( '\d+', x ).group(0) ) / 100 if pd.notnull( x ) else x )  
                
# final join
df_aux = df_aux.groupby( 'product_id' ).max().reset_index().fillna( 0 )
df_data = pd.merge( df_data, df_aux, on='product_id', how='left' )                     
                       
# drop columns  ##remove after generating the columns size_number e size_model
df_data = df_data.drop( columns=['size', 'product_safety', 'composition'], axis=1 )

# drop duplicates 
df_data = df_data.drop_duplicates()
df_data.shape



C:\Users\ferki\AppData\Local\Temp\ipykernel_9156\966745255.py:19: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_data['product_price'] = df_data['product_price'].str.replace( '$', '' )


(125, 13)

In [11]:
df_data.head()

,product_name,product_price,product_id,fit,color_name,style_id,color_id,scrapy_datetime,size_number,size_model,cotton,polyester,spandex
0,relaxed_jeans,39.99,0985159001,skinny_fit,black,0985159,001,2022-10-09 07:57:19,185,31/32,1.00,0.00,0.01
7,relaxed_jeans,39.99,0985159002,skinny_fit,denim_blue,0985159,002,2022-10-09 07:57:19,NaN,NaN,1.00,0.00,0.01
13,relaxed_jeans,39.99,0985159003,skinny_fit,dark_gray,0985159,003,2022-10-09 07:57:19,NaN,NaN,0.99,0.63,0.01
20,relaxed_jeans,39.99,0985159004,skinny_fit,light_denim_blue,0985159,004,2022-10-09 07:57:19,183,31/32,1.00,0.00,0.01
26,relaxed_jeans,39.99,0985159005,skinny_fit,dark_blue,0985159,005,2022-10-09 07:57:19,NaN,NaN,1.00,0.00,0.01


In [12]:
df_data.shape

(125, 13)

In [20]:
df_data.tail()

,product_name,product_price,product_id,fit,color_name,style_id,color_id,scrapy_datetime,size_number,size_model,cotton,polyester,spandex
1742,relaxed_jeans,39.99,0875105017,relaxed_fit,dark_denim_blue,0875105,017,2022-10-09 07:57:19,185,31/32,1.0,0.65,0.0
1749,relaxed_jeans,39.99,0875105018,relaxed_fit,pale_denim_blue,0875105,018,2022-10-09 07:57:19,187,31/32,1.0,0.65,0.0
1756,relaxed_jeans,39.99,0875105023,relaxed_fit,black,0875105,023,2022-10-09 07:57:19,NaN,NaN,1.0,0.65,0.0
1763,relaxed_jeans,39.99,0875105024,relaxed_fit,light_denim_blue,0875105,024,2022-10-09 07:57:19,185,31/32,1.0,0.65,0.0
1770,NaN,NaN,0875105026,relaxed_fit,light_gray,0875105,026,2022-10-09 07:57:19,NaN,NaN,1.0,0.00,0.0


# 3.0 Data Insert 

In [48]:
data_insert = df_data[[
    'product_id',
    'style_id',
    'color_id',
    'product_name',
    'color_name',
    'fit',
    'product_price',
    'size_number',
    'size_model',
    'cotton',
    'polyester',
    'spandex',
    'scrapy_datetime' 
]] 

In [49]:
query_showroom_schema = """
    CREATE TABLE vitrine(
        product_id      TEXT,
        style_id        TEXT,
        color_id        TEXT,
        product_name    TEXT,
        color_name      TEXT,
        fit             TEXT,
        product_price   REAL,
        size_number     TEXT,
        size_model      TEXT,
        cotton          REAL,
        polyester       REAL,
        spandex         REAL,
        scrapy_datetime TEXT 

    )
"""

In [50]:
# query_drop = """
# DROP TABLE vitrine
# """

In [52]:
# create table 
conn = sqlite3.connect( 'database_hm.sqlite' )
#cursor = conn.execute( query_drop )
cursor = conn.execute( query_showroom_schema )
conn.commit()
#conn.close()

In [53]:
# create database connection 
conn = create_engine( 'sqlite:///database_hm.sqlite', echo=False )

# data insert 
data_insert.to_sql( 'vitrine', con=conn, if_exists='append', index=False )

125

In [54]:
query = """
    SELECT * FROM vitrine
"""

df = pd.read_sql_query( query, conn)
df.head()

,product_id,style_id,color_id,product_name,color_name,fit,product_price,size_number,size_model,cotton,polyester,spandex,scrapy_datetime
0,0985159001,0985159,001,relaxed_jeans,black,skinny_fit,39.99,185,31/32,1.00,0.00,0.01,2022-10-09 07:57:19
1,0985159002,0985159,002,relaxed_jeans,denim_blue,skinny_fit,39.99,None,None,1.00,0.00,0.01,2022-10-09 07:57:19
2,0985159003,0985159,003,relaxed_jeans,dark_gray,skinny_fit,39.99,None,None,0.99,0.63,0.01,2022-10-09 07:57:19
3,0985159004,0985159,004,relaxed_jeans,light_denim_blue,skinny_fit,39.99,183,31/32,1.00,0.00,0.01,2022-10-09 07:57:19
4,0985159005,0985159,005,relaxed_jeans,dark_blue,skinny_fit,39.99,None,None,1.00,0.00,0.01,2022-10-09 07:57:19


In [55]:
df.shape

(125, 13)